In [1]:
import os

In [2]:
%pwd

'c:\\Users\\LENOVO\\1. Projects\\Bodycode-Anomaly-Detection-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\LENOVO\\1. Projects\\Bodycode-Anomaly-Detection-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from bodycoteAnomalyDetection.constants import *
from bodycoteAnomalyDetection.utils.common import read_yaml, create_directories

[2024-01-04 10:24:27,566: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [12]:
# Define all your imports at the beginning
import glob
import os
import pandas as pd
from bodycoteAnomalyDetection.logging import logger
from datasets import load_dataset, load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def data_transform(self, example_batch):
        # Convert TimeStamp column to datetime format
        example_batch['TimeStamp'] = pd.to_datetime(example_batch['TimeStamp'], format='%d/%m/%Y %H:%M:%S')
        example_batch.drop_duplicates(subset='TimeStamp', keep='last', inplace=True)
        
        # Make time stamp as index and sort by the index
        example_batch.set_index('TimeStamp', inplace=True)
        example_batch.sort_index(inplace=True)
        
        # Return numeric dataset
        data_training = example_batch.select_dtypes(include=['float64', 'int64'])

        return example_batch, data_training
    
    def read_csv(self, file_path):
        return pd.read_csv(file_path)

    def convert(self, file_path):
        df = self.read_csv(file_path)
        clean_data, data_training = self.data_transformation(df)
        clean_data.to_csv(os.path.join(self.config.root_dir, 'clean_data.csv'))
        data_training.to_csv(os.path.join(self.config.root_dir, 'data_training.csv'))

    def process_all_csv(self):
        try:
            csv_files = glob.glob(os.path.join(self.config.data_path, '*.csv'))
            for file in csv_files:
                self.convert(file)
        except Exception as e:
            logger.error(f"An error occurred: {e}")
            raise

In [13]:
#### Pipeline
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.process_all_csv()
except Exception as e:
    logger.error(f"An error occurred in the pipeline: {e}")
    raise

[2024-01-04 10:24:30,502: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-04 10:24:30,505: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-04 10:24:30,507: INFO: common: created directory at: artifacts]
[2024-01-04 10:24:30,510: INFO: common: created directory at: artifacts/data_transformation]
